<a href="https://colab.research.google.com/github/kpenagos/03MAIR--Algoritmos--de--Optimizacion--2019/blob/master/AG2/Karen_Penagos_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Karen Isabella Penagos Actividad Guiada 2
https://colab.research.google.com/drive/1cCCfIbiOnTQVAxpxoaKZjGUX6xjN1G4u

In [0]:
import itertools
from functools import wraps

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

Viaje por el Rio Programación Dinámica

In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]


def precios(TARIFAS):
  #Total de nodos
  N = len(TARIFAS[0])
 
  #Inicializacion de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N]
  RUTA = [ [""]*N for i in [""]*N]
 
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
     
      for k in range(i,j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
          MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j])
          RUTA[i][j] =  k
        PRECIOS[i][j] = MIN
 
  return PRECIOS, RUTA

PRECIOS, RUTA = precios(TARIFAS)
print(PRECIOS)
print(RUTA)


def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta] )) + ' - ' + str( RUTA[desde][hasta] )
 

calcular_ruta(RUTA, 1, 6)


[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


'1 - 1 - 4'

***Problema de Asignación de Tareas con el método de Ramificación y Poda***

In [0]:
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [0]:
 # Función de coste
def valor(S, COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i]
  return VALOR

valor((0,1,2,3), COSTES)

73

In [0]:

# Resolvemos el problema Por fuerza bruta

#@calcular_tiempo
def fuerza_bruta(COSTES):
  mejor_valor = 10e10
  mejor_solucion = ''
 
  for S in list(itertools.permutations(range(len(COSTES)))):
    valor_tmp = valor(S,COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = S
     
  print('La mejor solucion es:', mejor_solucion, 'con valor:', mejor_valor)
 
fuerza_bruta(COSTES)

La mejor solucion es: (0, 3, 1, 2) con valor: 61


In [0]:
# Funcion para estimar un coste inferior para una solucion parcial

def CI(S, COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]
   
  #Estimaciones
  for i in range( len(COSTES)):
    if i not in S:
      VALOR += min ([ COSTES[j][i] for j in range(len(S), len(COSTES)) ])
     
  return VALOR
 
CI((1,2), COSTES)            


59

In [0]:
# Funcion para ramificar

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append( {'s': NODO+ (i,)  })
     
  return HIJOS

crear_hijos((0,2), 4)

[{'s': (0, 2, 1)}, {'s': (0, 2, 3)}]

In [0]:


def ramificacion_y_poda(COSTES):
  DIMENSION = len(COSTES)
  MEJOR_SOLUCION = tuple( i for i in range(DIMENSION) )
  CotaSup = valor(MEJOR_SOLUCION, COSTES)
 
  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((), COSTES) })
 
  iteracion=0
 
  while(len(NODOS) > 0):
    iteracion +=1
    print('\n#', iteracion)
       
    nodo_prometedor = min(NODOS, key=lambda x:x['ci'] )
   
    #Hijos
    HIJOS = [{'s':x['s'], 'ci':CI(x['s'], COSTES) } for x in crear_hijos(nodo_prometedor['s'], DIMENSION)]
   
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) > 0:
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL[0]
   
   
    #Poda
    HIJOS = [ x for x in HIJOS if x['ci'] < CotaSup ]
   
    print(nodo_prometedor)
    #Eliminamos el nodo expandido
    NODOS = [x for x in NODOS if x['s'] != nodo_prometedor['s'] ]
   
    #Añadimos los hijos
    NODOS.extend(HIJOS)
   
    print(NODOS)
   
  print('La mejor solucion es:', MEJOR_SOLUCION)
   
    #return HIJOS
   
ramificacion_y_poda(COSTES)  


# 1
{'s': (), 'ci': 58}
[{'s': (0,), 'ci': 60}, {'s': (1,), 'ci': 58}, {'s': (2,), 'ci': 65}]

# 2
{'s': (1,), 'ci': 58}
[{'s': (0,), 'ci': 60}, {'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 2), 'ci': 59}, {'s': (1, 3), 'ci': 64}]

# 3
{'s': (1, 2), 'ci': 59}
[{'s': (0,), 'ci': 60}, {'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}]

# 4
{'s': (0,), 'ci': 60}
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}, {'s': (0, 1), 'ci': 68}, {'s': (0, 2), 'ci': 61}, {'s': (0, 3), 'ci': 66}]

# 5
{'s': (0, 2), 'ci': 61}
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}, {'s': (0, 1), 'ci': 68}, {'s': (0, 3), 'ci': 66}, {'s': (0, 2, 1), 'ci': 69}, {'s': (0, 2, 3), 'ci': 61}]

# 6
{'s': (0, 2, 3), 'ci': 61}
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 6